# Простая база данных

Запись датафрейма в базу данных

In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv('keywords.csv')
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [3]:
con = sqlite3.connect('keywords.db')

In [4]:
df.to_sql('keywords', con, if_exists='replace')

In [5]:
con.close()

Чтение из базы

In [6]:
con = sqlite3.connect('keywords.db')

In [7]:
df_sql = pd.read_sql('select * from keywords where keyword like "%вконтакте%";', con)
df_sql

,index,keyword,shows
0,4,вконтакте,21014195
1,14,вконтакте моя страница,5971451
2,317,вконтакте вход на страницу,488442
3,530,моя страница вконтакте,5971451
4,896,вконтакте социальная сеть,202480
...,...,...,...
84,96819,оренбург онлайн вконтакте,3894
85,97029,раскрутка групп в вконтакте,3782
86,98809,весь арзамас вконтакте,3623
87,99696,моя страница вконтакте одноклассники,16548


Построчная обработка

In [8]:
cur = con.cursor()

In [9]:
cur.execute('select * from keywords where keyword like "%вконтакте%";')

In [10]:
line = cur.fetchone()
line

(4, 'вконтакте', 21014195)

In [11]:
another_line = cur.fetchone()
another_line

(14, 'вконтакте моя страница', 5971451)

In [12]:
for i, line in enumerate(cur.execute('select * from keywords where keyword like "%вконтакте%";')):
    print(line)
    
    if i > 5:
        break

(4, 'вконтакте', 21014195)
(14, 'вконтакте моя страница', 5971451)
(317, 'вконтакте вход на страницу', 488442)
(530, 'моя страница вконтакте', 5971451)
(896, 'вконтакте социальная сеть', 202480)
(1003, 'вконтакте вход', 190587)
(1093, 'вконтакте моя', 173001)


# Выполнение в несколько потоков

In [13]:
from multiprocessing import Pool, current_process

### Пример работы с Pool

In [14]:
import time

In [15]:
def process_step(step):
    print('Начинаю шаг {}, worker {}'.format(step, current_process()))
    time.sleep(1)
    print('Закончил шаг {}'.format(step))

In [16]:
%%time
for step in range(10):
    process_step(step)

Начинаю шаг 0, worker <_MainProcess(MainProcess, started)>
Закончил шаг 0
Начинаю шаг 1, worker <_MainProcess(MainProcess, started)>
Закончил шаг 1
Начинаю шаг 2, worker <_MainProcess(MainProcess, started)>
Закончил шаг 2
Начинаю шаг 3, worker <_MainProcess(MainProcess, started)>
Закончил шаг 3
Начинаю шаг 4, worker <_MainProcess(MainProcess, started)>
Закончил шаг 4
Начинаю шаг 5, worker <_MainProcess(MainProcess, started)>
Закончил шаг 5
Начинаю шаг 6, worker <_MainProcess(MainProcess, started)>
Закончил шаг 6
Начинаю шаг 7, worker <_MainProcess(MainProcess, started)>
Закончил шаг 7
Начинаю шаг 8, worker <_MainProcess(MainProcess, started)>
Закончил шаг 8
Начинаю шаг 9, worker <_MainProcess(MainProcess, started)>
Закончил шаг 9
CPU times: user 12.6 ms, sys: 4.59 ms, total: 17.2 ms
Wall time: 10 s


In [17]:
%%time
with Pool(processes=10) as p:
    p.map(process_step, range(10))

Начинаю шаг 0, worker <ForkProcess(ForkPoolWorker-1, started daemon)>
Начинаю шаг 2, worker <ForkProcess(ForkPoolWorker-3, started daemon)>
Начинаю шаг 1, worker <ForkProcess(ForkPoolWorker-2, started daemon)>
Начинаю шаг 3, worker <ForkProcess(ForkPoolWorker-4, started daemon)>
Начинаю шаг 4, worker <ForkProcess(ForkPoolWorker-5, started daemon)>
Начинаю шаг 5, worker <ForkProcess(ForkPoolWorker-6, started daemon)>
Начинаю шаг 7, worker <ForkProcess(ForkPoolWorker-8, started daemon)>
Начинаю шаг 6, worker <ForkProcess(ForkPoolWorker-7, started daemon)>
Начинаю шаг 8, worker <ForkProcess(ForkPoolWorker-9, started daemon)>
Начинаю шаг 9, worker <ForkProcess(ForkPoolWorker-10, started daemon)>
Закончил шаг 1
Закончил шаг 3
Закончил шаг 2
Закончил шаг 0
Закончил шаг 4
Закончил шаг 6
Закончил шаг 5
Закончил шаг 8
Закончил шаг 9
Закончил шаг 7
CPU times: user 26.7 ms, sys: 40.9 ms, total: 67.6 ms
Wall time: 1.22 s


### Стемминг через Pool

In [18]:
import time
from nltk.stem.snowball import SnowballStemmer

In [19]:
def process_line(line):
    keyword, shows = line.strip().split(',')
    return [stemmer.stem(word) for word in keyword.split(' ')]

In [20]:
%%time
stemmer = SnowballStemmer("russian")
result = []

with open('keywords.csv') as f:
    result = [process_line(line) for line in f]
        
result[:10]

CPU times: user 16.7 s, sys: 196 ms, total: 16.9 s
Wall time: 19.3 s


[['keyword'],
 ['вк'],
 ['одноклассник'],
 ['порн'],
 ['ютуб'],
 ['вконтакт'],
 ['одноклассник', 'мо', 'страниц'],
 ['майл'],
 ['авит'],
 ['переводчик']]

In [21]:
%%time
f = open('keywords.csv')
stemmer = SnowballStemmer("russian")

with Pool(5) as p:
    result = p.map(process_line, f)
    
result[:10]

CPU times: user 290 ms, sys: 114 ms, total: 405 ms
Wall time: 10.8 s


[['keyword'],
 ['вк'],
 ['одноклассник'],
 ['порн'],
 ['ютуб'],
 ['вконтакт'],
 ['одноклассник', 'мо', 'страниц'],
 ['майл'],
 ['авит'],
 ['переводчик']]

# Сложные объединения
Необходимо объединить таблицы left_table и right_table с учетом условий:
- если числовое значение из right_table укладывается в интервал из left_table, то добавляем столбец из right_table
- если значение из right_table вне интервала из left_table, но меньше 200, то подставить 0
- если значение из right_table больше 200 или нет соответствия, то исключить строчку из результата полностью
- таблица right_table влазит в RAM, левая - нет

In [22]:
left_table = [
    ['a', 10, 20],
    ['b', 30, 40],
    ['c', 50, 60],
]

In [23]:
right_table = [
    ['a', 15],
    ['b', 100],
    ['c', 300],
]

In [24]:
right_dict = {rec[0]: rec[1] for rec in right_table}
right_dict

{'a': 15, 'b': 100, 'c': 300}

In [25]:
def apply_rules(left_table_line):
    key, left_border, right_border = left_table_line
    
    # условие 3 (если есть соответствие)
    if key in right_dict:
        value_from_right_table = right_dict[key]
        
        # условие 3 (если значение < 200)
        if value_from_right_table < 200:
            if left_border < value_from_right_table < right_border:
                return value_from_right_table
            else:
                return 0

In [26]:
for line in left_table:
    right_value = apply_rules(line)
    if right_value is not None:
        line.append(right_value)
        print(line)

['a', 10, 20, 15]
['b', 30, 40, 0]
